In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import itertools
import scipy.io as sio
#np.set_printoptions(threshold=np.inf)
import csv

In [ ]:
slp03_matfile = "slp03m"

In [ ]:
#variable for sensor data
ecg = []
bp = []
eeg = []
resp = []

#variable for annotation data
time = []
sample_num = []


In [ ]:
input_vec_size = lstm_size = 4 # ECG, BP, EEG, Resp
time_step_size = 1*250*30 # 250sample/sec *30sec 
batch_size = 1 # 1dimension

In [ ]:
def mat_read(filename, num_data, ecg, bp, eeg, resp):
    mat_contents = sio.loadmat('data/'+filename+'.mat')
    val = mat_contents['val']
    ecg.append(val[0])
    bp.append(val[1])
    eeg.append(val[2])
    resp.append(val[3])
    for i in range(1,num_data):
        mat_contents = sio.loadmat('data/'+filename+' (%d).mat'% i)
        val = mat_contents['val']
        ecg.append(val[0])
        bp.append(val[1])
        eeg.append(val[2])
        resp.append(val[3])

In [ ]:
def anno_read(filename, sample, )

In [ ]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.01))

def model(X, W, B, lstm_size):
    # X, input shape: (batch_size, time_step_size, input_vec_size)
    XT = tf.transpose(X, [1, 0, 2])  # permute time_step_size and batch_size
    # XT shape: (time_step_size, batch_size, input_vec_size)
    XR = tf.reshape(XT, [-1, lstm_size]) # each row has input for each lstm cell (lstm_size=input_vec_size)
    # XR shape: (time_step_size * batch_size, input_vec_size)
    X_split = tf.split(0, time_step_size, XR) # split them to time_step_size (28 arrays)
    # Each array shape: (batch_size, input_vec_size)

    # Make lstm with lstm_size (each input vector size)
    lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size, forget_bias=1.0, state_is_tuple=True)

    # Get lstm cell output, time_step_size (28) arrays with lstm_size output: (batch_size, lstm_size)
    outputs, _states = tf.nn.rnn(lstm, X_split, dtype=tf.float32)

    # Linear activation
    # Get the last output
    return tf.matmul(outputs[-1], W) + B, lstm.state_size # State size to initialize the stat

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28)
teX = teX.reshape(-1, 28, 28)

X = tf.placeholder("float", [None, 28, 28])
Y = tf.placeholder("float", [None, 10])

# get lstm_size and output 10 labels
W = init_weights([lstm_size, 10])
B = init_weights([10])

py_x, state_size = model(X, W, B, lstm_size)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [ ]:
session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

# Launch the graph in a session
with tf.Session(config=session_conf) as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX)+1, batch_size)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

        test_indices = np.arange(len(teX))  # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices]})))

In [ ]:
mat_read(slp03_matfile, 6, ecg, bp, eeg, resp)
ecg = list(itertools.chain(*ecg))
bp = list(itertools.chain(*bp))
eeg = list(itertools.chain(*eeg))
resp = list(itertools.chain(*resp))

In [ ]:
print(len(ecg))

In [ ]:
sleep_stage_label = ['W','R','1','2','3','4'] #id-> char
sleep_stage_dic = {w: i for i, w in enumerate(sleep_stage_label)} # char->id
#x_data = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,1,0]],dtype = 'f') #hell
#sample = [char_dic[c] for c in "hello"] # to index